In [41]:
# Importing necessary libraries to extract the data properly from Postgres using PySpark
import pandas as pd
import numpy as np
from API_Key import username, password
import requests
import time
import psycopg2 as pg2
import json
from pandas.io.json import json_normalize
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as psf
import pyspark.sql.types as pst
from scipy.stats import pointbiserialr

In [2]:
spark = SparkSession \
    .builder \
    .appName("TFT Analysis") \
    .config("spark.jars", "/Users/hiowatah/downloads/postgresql-42.2.18.jar") \
    .master("local[10]") \
    .getOrCreate()

In [3]:
df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/TFT") \
    .option("dbtable", "matches") \
    .option("user", username) \
    .option("password", password) \
    .option("driver", "org.postgresql.Driver") \
    .load()

In [4]:
df.schema

StructType(List(StructField(match,StringType,true),StructField(game_datetime,LongType,true),StructField(game_length,DoubleType,true),StructField(game_version,StringType,true),StructField(gold_left,IntegerType,true),StructField(last_round,IntegerType,true),StructField(level,IntegerType,true),StructField(placement,IntegerType,true),StructField(puuid,StringType,true),StructField(time_eliminated,DoubleType,true),StructField(total_damage,IntegerType,true),StructField(traits,StringType,true),StructField(units,StringType,true)))

In [5]:
def jsonColParser(df, *cols, sanitize=True):
    res = df
    for i in cols:
        if sanitize:
            res = (
                res.withColumn(
                    i,
                    psf.concat(psf.lit('{"data": '), i, psf.lit('}'))
                )
            )
        schema = spark.read.json(res.rdd.map(lambda x: x[i])).schema
        res = res.withColumn(i, psf.from_json(psf.col(i), schema))
        
        if sanitize:
            res = res.withColumn(i, psf.col(i).data)
            
    return res

## Creating second dataframe of exploded columns because each player is going to take up multiple rows of data for each unit and trait they had for the game. Analysis will be easier by filtering the first dataframe and getting the match info from the second dataframe

In [6]:
df = jsonColParser(df, "units", "traits")

In [7]:
df.count()

140816

In [8]:
df.corr("placement", "total_damage")

-0.7804422281556648

In [9]:
df = df.where("game_version LIKE '%Version 11.2%'")

In [10]:
df.count()

52416

In [11]:
df = df.withColumn("unit_names", df.units.character_id)\
        .withColumn("units_tier", df.units.tier)\
        .withColumn("chosen", df.units.chosen)\
        .withColumn("items", df.units.items)\
        .withColumn("trait_name", df.traits['name'])\
        .withColumn("num_units_trait", df.traits.num_units)\
        .withColumn("trait_tier", df.traits.tier_current)\
        .withColumn("style", df.traits.style)\
        .withColumn("trait_tier_total", df.traits.tier_total)\
        .drop("units", "traits")

In [12]:
df.count()

52416

In [47]:
df.toPandas().to_excel("matches.xlsx")

In [50]:
df.withColumn("Champion_1", df['unit_names'].getItem(0))\
    .withColumn("Champion_2", df['unit_names'].getItem(1))\
    .withColumn("Champion_3", df['unit_names'].getItem(2))\
    .withColumn("Champion_4", df['unit_names'].getItem(3))\
    .withColumn("Champion_5", df['unit_names'].getItem(4))\
    .withColumn("Champion_6", df['unit_names'].getItem(5))\
    .withColumn("Champion_7", df['unit_names'].getItem(6))\
    .withColumn("Champion_8", df['unit_names'].getItem(7))\
    .withColumn("Champion_9", df['unit_names'].getItem(8))\
    .withColumn("Champion_10", df['unit_names'].getItem(9))\
    .withColumn("Champion_1_tier", df['units_tier'].getItem(0))\
    .withColumn("Champion_2_tier", df['units_tier'].getItem(1))\
    .withColumn("Champion_3_tier", df['units_tier'].getItem(2))\
    .withColumn("Champion_4_tier", df['units_tier'].getItem(3))\
    .withColumn("Champion_5_tier", df['units_tier'].getItem(4))\
    .withColumn("Champion_6_tier", df['units_tier'].getItem(5))\
    .withColumn("Champion_7_tier", df['units_tier'].getItem(6))\
    .withColumn("Champion_8_tier", df['units_tier'].getItem(7))\
    .withColumn("Champion_9_tier", df['units_tier'].getItem(8))\
    .withColumn("Champion_10_tier", df['units_tier'].getItem(9))\

+--------------+-------------+----------------+--------------------+---------+----------+-----+---------+--------------------+-----------------+------------+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+
|         match|game_datetime|     game_length|        game_version|gold_left|last_round|level|placement|               puuid|  time_eliminated|total_damage|          unit_names|          units_tier|         chosen|               items|          trait_name|     num_units_trait|          trait_tier|               style|    trait_tier_total| Champion_1|Champion_2| Champion_3|  Champion_4|Champion_5|Champion_6|Champion_7|Champion_8|Champion_9|
+--------------+-------------+----------------+--------------------+---------+----------+-----+---------+-----

In [34]:
df.select("*").withColumn("gold", psf.array_contains(psf.col('style'), 2)).where("gold = 'true'").show(1)

+--------------+-------------+----------------+--------------------+---------+----------+-----+---------+--------------------+------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----+
|         match|game_datetime|     game_length|        game_version|gold_left|last_round|level|placement|               puuid|   time_eliminated|total_damage|          unit_names|          units_tier|              chosen|               items|          trait_name|     num_units_trait|          trait_tier|               style|    trait_tier_total|gold|
+--------------+-------------+----------------+--------------------+---------+----------+-----+---------+--------------------+------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------

In [27]:
df.printSchema()

root
 |-- match: string (nullable = true)
 |-- game_datetime: long (nullable = true)
 |-- game_length: double (nullable = true)
 |-- game_version: string (nullable = true)
 |-- gold_left: integer (nullable = true)
 |-- last_round: integer (nullable = true)
 |-- level: integer (nullable = true)
 |-- placement: integer (nullable = true)
 |-- puuid: string (nullable = true)
 |-- time_eliminated: double (nullable = true)
 |-- total_damage: integer (nullable = true)
 |-- unit_names: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- units_tier: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- chosen: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- items: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: long (containsNull = true)
 |-- trait_name: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- num_units_trait: array (nullable = true)
 |  